In [2]:
import pandas as pd
from gluonts.dataset.common import ListDataset
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
from gluonts.evaluation import make_evaluation_predictions, Evaluator
import mxnet as mx

ModuleNotFoundError: No module named 'gluonts.model.deepar'

In [ ]:
file_path_weather = 'C:\Theo\School\ESIEE\Annee\E5\TimeSeries\Projet DeepAR\Dataset\weather\weather.csv'

df_weather = pd.read_csv(file_path_weather)

# Select features
features = ['p (mbar)', 'T (degC)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)', 'VPmax (mbar)',
            'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)', 'wv (m/s)', 'max. wv (m/s)', 
            'wd (deg)', 'rain (mm)', 'raining (s)', 'SWDR (W/m²)', 'PAR (µmol/m²/s)', 
            'max. PAR (µmol/m²/s)', 'Tlog (degC)', 'OT']

# Fill missing values
df = df.fillna(method='ffill')

# Create the training dataset
training_data = ListDataset(
    [{"start": df['date'].iloc[0], "target": df[feature].values} for feature in features],
    freq="1H"
)

# Create DeepAR estimator
estimator = DeepAREstimator(
    freq="1H",  # Hourly data
    prediction_length=24,  # Predict the next 24 hours
    context_length=48,  # Look back 48 hours
    trainer=Trainer(epochs=50, learning_rate=1e-3, batch_size=64, ctx=mx.cpu())
)

# Train the model
predictor = estimator.train(training_data=training_data)

# Generate predictions
forecast_it, ts_it = make_evaluation_predictions(
    dataset=training_data,
    predictor=predictor,
    num_samples=100
)

# Evaluate the model
evaluator = Evaluator()
forecasts = list(forecast_it)
tss = list(ts_it)
agg_metrics, item_metrics = evaluator(tss, forecasts)
print(f"MAE: {agg_metrics['MAE']}")
print(f"RMSE: {agg_metrics['RMSE']}")